In [9]:
# import necessary packages
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
import numpy as np

import seaborn as sns
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, preprocessing, svm
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from collections import Counter
import random

In [10]:
#import cleaned dataset from last project
data = pd.DataFrame.from_csv('cleaned.csv')

# convert all features to categorical integer values
#enc = LabelEncoder()
#for i in data.columns:
    #data[i] = enc.fit_transform(data[i])

BsmtQual = data['BsmtQual']
encodings = {'Ex': 6, 'Gd': 5, 'TA': 4, 'Fa': 3, 'Po': 2, 'NP' :1}
for key in encodings:
    BsmtQual = BsmtQual.replace(key, encodings[key])
    data['BsmtQual'] = BsmtQual
    
ExterQual = data['ExterQual']
encodings = {'Ex': 6, 'Gd': 5, 'TA': 4, 'Fa': 3, 'Po': 2, 'NP' :1}
for key in encodings:
    ExterQual = ExterQual.replace(key, encodings[key])
    data['ExterQual'] = ExterQual
    
ExterCond = data['ExterCond']
encodings = {'Ex': 6, 'Gd': 5, 'TA': 4, 'Fa': 3, 'Po': 2, 'NP' :1}
for key in encodings:
    ExterCond = ExterCond.replace(key, encodings[key])
    data['ExterCond'] = ExterCond
    
BsmtCond = data['BsmtCond']
encodings = {'Ex': 6, 'Gd': 5, 'TA': 4, 'Fa': 3, 'Po': 2, 'NP' :1}
for key in encodings:
    BsmtCond = BsmtCond.replace(key, encodings[key])
    data['BsmtCond'] = BsmtCond
    
KitchenQual = data['KitchenQual']
encodings = {'Ex': 6, 'Gd': 5, 'TA': 4, 'Fa': 3, 'Po': 2, 'NP' :1}
for key in encodings:
    KitchenQual = KitchenQual.replace(key, encodings[key])
    data['KitchenQual'] = KitchenQual
    
GarageQual = data['GarageQual']
encodings = {'Ex': 6, 'Gd': 5, 'TA': 4, 'Fa': 3, 'Po': 2, 'NP' :1}
for key in encodings:
    GarageQual = GarageQual.replace(key, encodings[key])
    data['GarageQual'] = GarageQual

In [11]:
# Add target variable column 'qualAboveAverage' to dataframe 
# Specifies if house quality is above agerage (1) or below average(0)
data['qualAboveAverage'] = (data['OverallQual'] > 5).astype(int)

# Store target variable (qualAboveAverage) as y
Y = data['qualAboveAverage']

columns = data.columns 
columns_dtypes = data.dtypes
numerical_types = [np.int64, np.int32, np.float32, np.float64]
valid_col = [columns[i] for i in range(len(columns)) if columns_dtypes[i] in numerical_types]
valid_data = pd.DataFrame(data, columns = valid_col)

In [12]:
for column in valid_data.columns.values:
    corr = abs(valid_data[column].corr(Y))
    if corr > .4:
        print(column + ':', corr)

OverallQual: 0.784865736046
YearBuilt: 0.479962376201
YearRemodAdd: 0.437861757046
ExterQual: 0.506235761975
BsmtQual: 0.501488389632
GrLivArea: 0.446058280118
FullBath: 0.487962875878
KitchenQual: 0.46337623216
GarageCars: 0.469083851474
SalePrice: 0.531738813349
qualAboveAverage: 1.0


In [13]:
# Calculate the baseline for Logistic Regression model
logit_baseline = max(Y.value_counts())/len(Y)
print('Logistic Regression Baseline Value:', logit_baseline)

Logistic Regression Baseline Value: 0.631434454358


In [14]:
#To choose appropriate features, correlation coeffient was used to choose columns. However, it did not yield  
#a desirable output. Therefore, the method below was used to find suitable features.
# Limit the feature set to intuitively relevant features 
#Houses that are newer are better quality; good neighborhood - better quality houses; better quality houses are 
#generally sold at higher prices.
#relevant = ['YearRemodAdd', 'YearBuilt', 'MSZoning', 'Neighborhood','SalePrice', 'Condition1', 'Condition2']
#X = data.filter(relevant)
#X = data.drop(['OverallQual', 'qualAboveAverage', 'LandSlope', 'RoofStyle', 'Fireplaces'], axis=1)
#X = data.drop('OverallQual', axis=1)
#X = data.drop('qualAboveAverage', axis = 1)
relevant = ['YearBuilt', 'YearRemodAdd', 'ExtreQul', 'BsmtQual', 'TotalBsmtSF', 'GrLivArea','FullBath', 'KitchenQual', 'GarageCars', 'GarageArea', 'SalePrice']        
X = valid_data.filter(relevant)
print(X)


      YearBuilt  YearRemodAdd  BsmtQual  TotalBsmtSF  GrLivArea  FullBath  \
Id                                                                          
1          2003          2003         5          856       1710         2   
2          1976          1976         5         1262       1262         2   
3          2001          2002         5          920       1786         2   
4          1915          1970         4          756       1717         1   
5          2000          2000         5         1145       2198         2   
6          1993          1995         5          796       1362         1   
7          2004          2005         6         1686       1694         2   
8          1973          1973         5         1107       2090         2   
9          1931          1950         4          952       1774         2   
10         1939          1950         4          991       1077         1   
11         1965          1965         4         1040       1040         1   

In [15]:
splits = [0.1, 0.2, 0.3, 0.4, 0.5]
random_states = [98, 1560, 999, 105002, 77393]

In [16]:
# build model and fit on train set
model = LogisticRegression()

for i in range(0,5):
    # Split dataset into training and test data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=splits[i], random_state=random_states[i])
    # Fit the model to the training data
    model.fit(X_train, Y_train)
    # Predict on the testing data
    model.predict(X_test)
    # Test the score of the model
    score = model.score(X_test, Y_test)
    # Calculate % improvement over the baseline
    improvement = 100*(score - logit_baseline)/logit_baseline
    print("The Logistic Regression for a train-test-split of {}-{} has an accuracy score of:\n{:.4f}"
          .format(1-splits[i], splits[i], score))
    print("Thus, this model is {:.2f}% more accurate than the baseline model."
          .format(improvement))
    print("\n")

The Logistic Regression for a train-test-split of 0.9-0.1 has an accuracy score of:
0.8425
Thus, this model is 33.42% more accurate than the baseline model.


The Logistic Regression for a train-test-split of 0.8-0.2 has an accuracy score of:
0.8390
Thus, this model is 32.88% more accurate than the baseline model.


The Logistic Regression for a train-test-split of 0.7-0.3 has an accuracy score of:
0.8151
Thus, this model is 29.08% more accurate than the baseline model.


The Logistic Regression for a train-test-split of 0.6-0.4 has an accuracy score of:
0.8302
Thus, this model is 31.48% more accurate than the baseline model.


The Logistic Regression for a train-test-split of 0.5-0.5 has an accuracy score of:
0.8258
Thus, this model is 30.78% more accurate than the baseline model.


